In [1]:
import duckdb

In [2]:
def query(q):
    db = duckdb.connect("../data/chess.duckdb")
    db.sql("SET TimeZone = 'UTC'")
    df = db.sql(q).to_df()
    db.close()
    return df

In [3]:
query('select * from information_schema.tables')

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,chess,chess_data_core,games,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,chess,chess_data_core,player_games,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,chess,chess_data_prep,game_moves,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,chess,chess_data_raw,chess_openings,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,chess,chess_data_raw,players_games,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,chess,chess_data_raw,players_profiles,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,chess,chess_data_raw,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,chess,chess_data_raw,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
8,chess,chess_data_raw,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None
9,chess,chess_data_staging,game_moves,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [7]:
query("select * from chess_data_raw.chess_openings where opening = 'unknown'")['url'][0]

'https://www.chess.com/openings/Nimzowitsch-Defense-Declined-2...Nf6'

In [20]:
query("""
    with dataset as (
        select 
        uuid
        , array_agg(move order by game_move_index) as lst
        from chess_data_staging.game_moves
        where uuid = '0019bf9d-62b3-11ef-9f40-a9fcd401000f'
        group by 1
    )
    select *, lst[1:2]
    from dataset
""")

,uuid,lst,lst[1:2]
0,0019bf9d-62b3-11ef-9f40-a9fcd401000f,"[e4, e5, Nf3, Qf6, Bc4, Bc5, O-O, d6, c3, Qg6,...","[e4, e5]"


In [19]:
query("""
    select 
    uuid,
    move
    from chess_data_staging.game_moves
    where uuid = '0019bf9d-62b3-11ef-9f40-a9fcd401000f'
    order by game_move_index
    limit 10
""")

,uuid,move
0,0019bf9d-62b3-11ef-9f40-a9fcd401000f,e4
1,0019bf9d-62b3-11ef-9f40-a9fcd401000f,e5
2,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Nf3
3,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Qf6
4,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Bc4
5,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Bc5
6,0019bf9d-62b3-11ef-9f40-a9fcd401000f,O-O
7,0019bf9d-62b3-11ef-9f40-a9fcd401000f,d6
8,0019bf9d-62b3-11ef-9f40-a9fcd401000f,c3
9,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Qg6


In [7]:
username = 'johnnywhoopp'

q =f"""
with player_games as (
    select
    end_time
    , url 
    , pgn
    , time_control
    , rated
    , tcn
    , uuid
    , initial_setup
    , fen
    , time_class
    , rules
    , white__rating
    , white__result
    , white__aid
    , white__username
    , white__uuid
    , black__rating
    , black__result
    , black__aid
    , black__username
    , black__uuid
    , accuracies__white
    , accuracies__black
    from chess_data_staging.players_games
)
, final as (
    select
    *
    
    -- Time control details
    , cast(split(time_control, '+')[1] as integer) as time_control_base
    , cast(coalesce(split(time_control, '+')[2], '0') as integer) as time_control_add_seconds
    
    -- PGN details
    , split(pgn, '\n\n')[2] as pgn_moves
    , regexp_extract_all(pgn_moves, '\d+\.+ [\S]+') as pgn_move_extract
    , regexp_extract_all(pgn_moves, '{{\[%clk \S+\]}}') as pgn_clock_extract
    
    -- PGN ECO details
    , regexp_extract(pgn, '(ECO )"(.*)"', 2) as eco
    , regexp_extract(pgn, '(ECOUrl )"(.*)"', 2) as eco_url
    , replace(eco_url, 'https://www.chess.com/openings/', '') as eco_name
    
    -- PLAYER details
    , if(lower(white__username)='{username}', 'White', 'Black') as player_color
    , if(player_color = 'White', white__rating, black__rating) as player_rating
    , if(player_color = 'White', white__result, black__result) as player_result
    
    -- OPPONENT DETAILS
    , if(player_color = 'White', black__rating, white__rating) as opponent_rating
    , if(player_color = 'White', black__result, white__result) as opponent_result
    
    -- PLAYER-OPPONENT DETAILS
    , opponent_rating > player_rating as is_stronger_opponent
    , case 
        when player_result = 'win' then 'win'
        when opponent_result = 'win' then 'lose'
        when player_result <> 'win' and opponent_result <> 'win' then 'draw'
        else 'unknown'
    end as player_wdl
    , if(player_result='win', opponent_result, player_result) as player_wdl_reason
    
    from player_games
    limit 10
)
select * from final
"""

query(q)

,end_time,url,pgn,time_control,rated,tcn,uuid,initial_setup,fen,time_class,...,eco_url,eco_name,player_color,player_rating,player_result,opponent_rating,opponent_result,is_stronger_opponent,player_wdl,player_wdl_reason
0,2024-05-01 09:54:57,https://www.chess.com/game/7bf31f6f-079f-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,bsYIpxZRoE2UnD92EM5QxF7ZmCXPgv6XiqZEfxEDltDwem...,7bf31f6f-079f-11ef-9338-b21a8d01000f,,4k3/8/5Q2/P1P1B2P/1P2P3/7B/2K5/6R1 b - -,rapid,...,https://www.chess.com/openings/Van-Geet-Openin...,Van-Geet-Opening-1...c5,Black,591,stalemate,600,stalemate,True,draw,stalemate
1,2024-05-02 02:22:41,https://www.chess.com/game/0b3e5e9f-082a-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC5Qgv0KfH7TlBWOHQTQBKQCcuZRKR9RbsC0eg6EuB1Tfe...,0b3e5e9f-082a-11ef-a262-7a436f01000f,,5rk1/1pp3pp/p7/1P2N3/P7/2R5/2P2PPP/3r2K1 w - -,rapid,...,https://www.chess.com/openings/Ruy-Lopez-Openi...,Ruy-Lopez-Opening-Gunderam-Variation,White,584,checkmated,621,win,True,lose,checkmated
2,2024-05-04 08:08:17,https://www.chess.com/game/b336281d-09eb-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCZRgv!TbsYQlBRJCKTCdmCsjs1TcD6EKT2Tmu92ft0KBK...,b336281d-09eb-11ef-8a15-f5237801000f,,5r1k/3Q4/7p/P2Bb2P/8/8/5PP1/2r3K1 w - -,rapid,...,https://www.chess.com/openings/Pirc-Defense-Cz...,Pirc-Defense-Czech-Defense-4.Nf3,White,576,checkmated,597,win,True,lose,checkmated
3,2024-05-04 11:50:54,https://www.chess.com/game/32f6a42f-0a0a-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIbs5QdN2UNIZRIu!Tnv92fH8!HQXQlB6Xgm7Zeg46BJ...,32f6a42f-0a0a-11ef-8a15-f5237801000f,,r7/1P6/1N3Q2/6pk/3B1p1p/P2K4/6PP/2R5 b - -,rapid,...,https://www.chess.com/openings/Closed-Sicilian...,Closed-Sicilian-Defense-Traditional-Line,Black,568,resigned,584,win,True,lose,resigned
4,2024-05-19 12:41:22,https://www.chess.com/game/52dc600d-15db-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIfA0Sbs!Tlt5QnDZRcu2Udm92ec8!pxRJCJSJArJBsJ...,52dc600d-15db-11ef-9d9d-af569601000f,,5rk1/p4p2/6p1/1p3bQp/6PK/3Pr1qP/2PR4/3R4 w - -,rapid,...,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Bowdler-Attack-2...e6-3.Nc3,Black,576,win,577,checkmated,True,win,checkmated
5,2024-05-21 03:18:19,https://www.chess.com/game/60480663-1720-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0KgvZRfA6Eeg7TAm5ZbsWOsJTVJY80Y4,60480663-1720-11ef-ac29-13086c01000f,,N4bnr/1p1nkppp/p2p3q/4p3/4P1b1/5N2/PPPPBPPP/R1...,rapid,...,https://www.chess.com/openings/Philidor-Defens...,Philidor-Defense-3.Bc4,White,584,win,578,resigned,False,win,resigned
6,2024-05-21 13:56:57,https://www.chess.com/game/9e9051f9-1777-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0Kgv5QfA!Tbs9Ieg8!ltZRcuIunu6EtBKBuBEvfvTEBJ...,9e9051f9-1777-11ef-a763-49c05101000f,,6k1/pp6/8/1Kq5/3q4/P7/8/8 w - -,rapid,...,https://www.chess.com/openings/Giuoco-Piano-Ga...,Giuoco-Piano-Game-Four-Knights-Game-5.O-O,White,576,checkmated,590,win,True,lose,checkmated
7,2024-05-21 14:30:39,https://www.chess.com/game/1e22df69-177c-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0Kgv5QfHQzegYQvKQHbsZRKv!TiqzQsHTClB7Pqy6Ecu...,1e22df69-177c-11ef-bd50-71056501000f,,1k6/pp2R1p1/1b6/6p1/P1r5/4RK2/1PP5/8 b - -,rapid,...,https://www.chess.com/openings/Ruy-Lopez-Opening,Ruy-Lopez-Opening,White,584,win,588,resigned,True,win,resigned
8,2024-05-22 07:36:34,https://www.chess.com/game/36e9234a-180b-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIgv5QlBIBvBQBdB!TCKTNfm2Ueg92bs8!cuZRfd2KBJ...,36e9234a-180b-11ef-b4d1-05170701000f,,2r1r1k1/3QBpb1/4p1pp/3pq3/1P6/8/P1R1BPPP/3R2K1...,rapid,...,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Open-3...cxd4-4.Nxd4,Black,576,timeout,607,win,True,lose,timeout
9,2024-05-22 13:44:02,https://www.chess.com/game/9f2573f4-1840-11ef-...,"[Event ""Live Chess""]\n[Site "